In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import SGDClassifier
import xgboost as xgb

In [2]:
%cd /Users/vladimirkalajcidi/vk_recsys

/Users/vladimirkalajcidi/vk_recsys


In [3]:
train_interactions = pd.read_parquet('data/train_interactions.parquet')
users_meta = pd.read_parquet('data/users_meta.parquet')
items_meta = pd.read_parquet('data/items_meta.parquet')

In [4]:
items_meta=items_meta.drop(columns=['embeddings'])

In [5]:
def calculate_unique_sources(df):
    """ Calculates the number of unique source_ids for each user_id in the given DataFrame. """
    user_sources = df.merge(items_meta[['item_id', 'source_id']], on='item_id', how='left')
    unique_source_count = user_sources.groupby('user_id')['source_id'].nunique().reset_index(name='unique_source_count')
    return unique_source_count

In [6]:
# Prepare DataFrames for storing item metrics and user metrics
item_metrics_final = pd.DataFrame()
user_metrics_final = pd.DataFrame()

# Define functions for metrics calculations
def calculate_item_metrics(df):
    item_metrics = df.groupby('item_id').agg(
        total_likes=('like', 'sum'),
        total_dislikes=('dislike', 'sum'),
        total_shares=('share', 'sum'),
        total_bookmarks=('bookmarks', 'sum'),
        avg_timespent=('timespent', 'mean'),
    ).reset_index()
    return item_metrics

def calculate_gender_metrics(df, users_meta):
    # Merge interactions with user metadata
    extended_df = df.merge(users_meta, on='user_id', how='left')

    # Calculate likes and dislikes for males (1) and females (2)
    gender_likes = extended_df[extended_df['like'] == 1].groupby(['item_id', 'gender']).size().unstack(fill_value=0)

    male_likes = gender_likes.get(1, pd.Series(0)).reset_index(name='male_likes')
    female_likes = gender_likes.get(2, pd.Series(0)).reset_index(name='female_likes')

    gender_dislikes = extended_df[extended_df['dislike'] == 1].groupby(['item_id', 'gender']).size().unstack(fill_value=0)

    male_dislikes = gender_dislikes.get(1, pd.Series(0)).reset_index(name='male_dislikes')
    female_dislikes = gender_dislikes.get(2, pd.Series(0)).reset_index(name='female_dislikes')

    # Merge the results
    male_likes = male_likes.rename(columns={male_likes.columns[0]: 'item_id'})
    female_likes = female_likes.rename(columns={female_likes.columns[0]: 'item_id'})
    male_dislikes = male_dislikes.rename(columns={male_dislikes.columns[0]: 'item_id'})
    female_dislikes = female_dislikes.rename(columns={female_dislikes.columns[0]: 'item_id'})

    return male_likes, female_likes, male_dislikes, female_dislikes

def calculate_unique_sources(df):
    user_sources = df.merge(items_meta[['item_id', 'source_id']], on='item_id', how='left')
    unique_source_count = user_sources.groupby('user_id')['source_id'].nunique().reset_index(name='unique_source_count')
    return unique_source_count


In [7]:
def calculate_metrics(train_interactions, users_meta, items_meta):
    # Step 1: Count likes and occurrences of each item_id
    likes_count = train_interactions[train_interactions['like'] == 1].groupby('item_id').size().reset_index(name='total_likes')
    item_encounter_count = train_interactions['item_id'].value_counts().reset_index()
    item_encounter_count.columns = ['item_id', 'item_encounter_count']
    
    # Step 2: Merge likes count with encounter count
    item_metrics = likes_count.merge(item_encounter_count, on='item_id', how='left').fillna(0)

    # Step 3: Count likes by gender
    gender_likes = (
        train_interactions[train_interactions['like'] == 1]
        .merge(users_meta[['user_id', 'gender']], on='user_id', how='left')
        .groupby(['item_id', 'gender'])
        .size()
        .unstack(fill_value=0)
    )

    # Prepare the gender like counts
    male_likes = gender_likes.get(1, pd.Series(0)).reset_index(name='male_likes')
    female_likes = gender_likes.get(2, pd.Series(0)).reset_index(name='female_likes')

    # Step 4: Merge gender likes into item metrics
    item_metrics = item_metrics.merge(male_likes, on='item_id', how='left').fillna(0)
    item_metrics = item_metrics.merge(female_likes, on='item_id', how='left').fillna(0)

    # Calculate the average likes by gender
    item_metrics['avg_likes_per_gender'] = (item_metrics['male_likes'] + item_metrics['female_likes']) / 2


    return item_metrics

# Load your data

In [8]:
# Call the function to calculate metrics and merge with items_meta
result = calculate_metrics(train_interactions, users_meta, items_meta)

# Display the result DataFrame (or save it)
print(result.head())

   item_id  total_likes  item_encounter_count  male_likes  female_likes  \
0        0            3                    51           0             3   
1        1            3                    43           0             3   
2        2            5                    63           1             4   
3        3            7                    97           0             7   
4        4           16                   192           0            16   

   avg_likes_per_gender  
0                   1.5  
1                   1.5  
2                   2.5  
3                   3.5  
4                   8.0  


In [9]:
# Loop through each specified row count
# Calculate item metrics
item_metrics = calculate_item_metrics(train_interactions)
male_likes, female_likes, male_dislikes, female_dislikes = calculate_gender_metrics(train_interactions, users_meta)

# Merge gender metrics
item_metrics = item_metrics.merge(male_likes, on='item_id', how='left')
item_metrics = item_metrics.merge(female_likes, on='item_id', how='left')
item_metrics = item_metrics.merge(male_dislikes, on='item_id', how='left')
item_metrics = item_metrics.merge(female_dislikes, on='item_id', how='left')
item_metrics = item_metrics.merge(result, on='item_id', how='left')

# Add suffix to distinguish metrics by row count

item_metrics.columns = [f'{col}_all' if col != 'item_id' else 'item_id' for col in item_metrics.columns]

# Merge into the final aggregated metrics DataFrame
if item_metrics_final.empty:
    item_metrics_final = item_metrics
else:
    item_metrics_final = item_metrics_final.merge(item_metrics, on='item_id', how='outer')

# Calculate unique source counts for users and merge into user metrics DataFrame
user_unique_sources = calculate_unique_sources(train_interactions)

# Add suffix for user metrics by row count
user_unique_sources.columns = ['user_id', f'unique_source_count_all']

if user_metrics_final.empty:
    user_metrics_final = user_unique_sources
else:
    user_metrics_final = user_metrics_final.merge(user_unique_sources, on='user_id', how='outer')


In [10]:
items_meta = items_meta.merge(item_metrics_final, on='item_id', how='left')
items_meta.fillna(0, inplace=True)

users_meta = users_meta.merge(user_metrics_final, on='user_id', how='left')
users_meta.fillna(0, inplace=True)

In [11]:
test_pairs = pd.read_csv('data/test_pairs.csv')

users = test_pairs['user_id'].unique()
items = test_pairs['item_id'].unique()

In [12]:
train_interactions['like'] = train_interactions['like'].astype('int32')    # Convert to signed integer
train_interactions['dislike'] = train_interactions['dislike'].astype('int32')  # Convert to signed integer

train_interactions['label'] = abs(train_interactions['like'] - train_interactions['dislike'])

#train_interactions = train_interactions[train_interactions['user_id'].isin(users)][train_interactions['item_id'].isin(items)]
train_interactions = train_interactions.loc[5_000_000:]

In [13]:
train_interactions = train_interactions.merge(users_meta, on='user_id', how='left')
train_interactions = train_interactions.merge(items_meta, on='item_id', how='left')

In [14]:
train_interactions = train_interactions[['gender', 'age', 'unique_source_count_all',
       'source_id', 'duration', 'total_likes_x_all', 'total_dislikes_all',
       'total_shares_all', 'total_bookmarks_all', 'avg_timespent_all',
       'male_likes_x_all', 'female_likes_x_all', 'male_dislikes_all',
       'female_dislikes_all', 'item_encounter_count_all',
       'avg_likes_per_gender_all', 'label']]

In [15]:
train_interactions.columns

Index(['gender', 'age', 'unique_source_count_all', 'source_id', 'duration',
       'total_likes_x_all', 'total_dislikes_all', 'total_shares_all',
       'total_bookmarks_all', 'avg_timespent_all', 'male_likes_x_all',
       'female_likes_x_all', 'male_dislikes_all', 'female_dislikes_all',
       'item_encounter_count_all', 'avg_likes_per_gender_all', 'label'],
      dtype='object')

In [16]:
len(train_interactions)

140667282

In [17]:
params = {
    'tree_method': 'exact',
    'objective': 'binary:logistic'
}
num_boost_round = 20
class_weight = len(train_interactions[train_interactions['label'] == 0]) / len(train_interactions[train_interactions['label'] == 1])

clf = xgb.XGBClassifier(n_estimators=num_boost_round, scale_pos_weight=class_weight, **params)
clf.fit(train_interactions.drop(columns=['label']), train_interactions['label'])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=20, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [21]:
test_pairs = pd.read_csv('data/test_pairs.csv')

test_pairs = test_pairs.merge(users_meta, on='user_id', how='left')
test_pairs = test_pairs.merge(items_meta, on='item_id', how='left')

test_pairs = test_pairs[['gender', 'age', 'unique_source_count_all', 'source_id', 'duration',
       'total_likes_x_all', 'total_dislikes_all', 'total_shares_all',
       'total_bookmarks_all', 'avg_timespent_all', 'male_likes_x_all',
       'female_likes_x_all', 'male_dislikes_all', 'female_dislikes_all',
       'item_encounter_count_all', 'avg_likes_per_gender_all']]

In [22]:
test_pairs.head()

,gender,age,unique_source_count_all,source_id,duration,total_likes_x_all,total_dislikes_all,total_shares_all,total_bookmarks_all,avg_timespent_all,male_likes_x_all,female_likes_x_all,male_dislikes_all,female_dislikes_all,item_encounter_count_all,avg_likes_per_gender_all
0,2,35,145,5119,9,2308,1,614,35,7.494987,558.0,1750.0,0.0,1.0,15659.0,1154.0
1,2,35,145,2720,5,1110,1,144,53,5.802824,180.0,930.0,0.0,1.0,6091.0,555.0
2,2,35,145,11601,32,379,1,101,10,21.974728,2.0,377.0,0.0,1.0,3126.0,189.5
3,2,35,145,4296,10,1532,2,463,8,8.337730,139.0,1393.0,1.0,1.0,14556.0,766.0
4,2,35,145,18812,38,134,2,52,4,26.483355,2.0,132.0,0.0,2.0,5317.0,67.0


In [23]:
result = clf.predict_proba(test_pairs)
result = [i[1] for i in result]

test_pairs = pd.read_csv('data/test_pairs.csv')
test_pairs['predict'] = result

test_pairs.to_csv('sub_lightgbm.csv', index=False)